<a href="https://colab.research.google.com/github/lessw2020/training-detr/blob/master/training_detr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Example of how to setup and train your own custom dataset using DETR
# In this notebook, we'll do fine tuning of the existing head
# For larger datasets (say > 10K images) training from scratch may be preferred and we'll do a different notebook for that



In [0]:
# 

# Setup PyTorch environment and DETR

In [6]:
!pip install torch torchvision


In [12]:
import torch; 
torch_version = torch.__version__
assert float(torch_version[:3]) >= 1.50   # make sure 1.50 or higher
torch_version


'1.5.0+cu101'

In [13]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'   #for coco apis


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-q3ujzc0c
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-q3ujzc0c
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=267007 sha256=71ba94d3a7ea07cb3b26f6ca417fed9f812a9ca10df5f164b19837c5c4925c9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-psf_833z/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.0
    Uninstalling pycocotools-2.0.0:
      Successfully uninstalled pycocotools-2.0.0


In [14]:
!git clone https://github.com/facebookresearch/detr.git


Cloning into 'detr'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 112 (delta 0), reused 0 (delta 0), pack-reused 108
Receiving objects: 100% (112/112), 7.27 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [15]:
from pathlib import Path as p; p.cwd()  # make sure we know where we are

PosixPath('/content')

In [25]:
!ls 

detr  sample_data


PosixPath('/content')

In [30]:
%cd /content/detr  
# move default dir to detr to begin...

/content/detr


In [31]:
!ls   # make sure everything looks good

datasets    hubconf.py	models		  run_with_submitit.py	util
Dockerfile  LICENSE	README.md	  test_all.py
engine.py   main.py	requirements.txt  tox.ini


# Create a new model with blank fine tuning section for your dataset


In [0]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
#from torchvision.models import resnet50
import torchvision.transforms as T

In [35]:
# download the latest pre-trained weights for detr with resnet50 backbone:
# detr resnet 50 backbone and head
!wget https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth


# Note that you want to check if this link becomes outdated (i.e. newer weights are available) 
# weights are located at:  https://github.com/facebookresearch/detr#model-zoo


# or use resnet 101...we'll use res50 for this colab:
# !wget https://dl.fbaipublicfiles.com/detr/detr-r101-2c7b67e5.pth  

# Further note - there is a DC5 model for both resnet50 and resnet101.  
# Both are a bit more accurate than their regular counterpart, but increase memory consumption 2x 
# and are hard to train as a result (CUDA out of memory errors)...we will use plain resnet50 here.

--2020-06-13 21:45:38--  https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166618694 (159M) [application/octet-stream]
Saving to: ‘detr-r50-e632da11.pth’

detr-r50-e632da11.p 100%[===================>] 158.90M  25.9MB/s    in 6.6s    

2020-06-13 21:45:46 (24.0 MB/s) - ‘detr-r50-e632da11.pth’ saved [166618694/166618694]

